In [61]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin
from tqdm import tqdm

import numpy as np
import pandas as pd

# from dtaidistance import dtw
import pywt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier 
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier

import torch

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from utils import *
from viz import *

# Hyperparameters

## EDA

In [3]:
PLOT = False

## PREPROCESSING

In [4]:
SMOOTHING_WINDOW = 1 # 1 for no smoothing
DOWNSAMPLE_WINDOW = 1 # 1 for no downsampling

## ML

# Get the data

In [5]:
data_folder = pjoin("data", "MixedShapesRegularTrain")
train_fname = "MixedShapesRegularTrain_TRAIN.tsv"
test_fname = "MixedShapesRegularTrain_TEST.tsv"

train_data = pd.read_csv(pjoin(data_folder, train_fname), sep='\t')

# EDA

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Columns: 1025 entries, 1 to 1.1120469
dtypes: float64(1024), int64(1)
memory usage: 3.9 MB


In [7]:
train_data.head()

,1,1.1309687,1.1204832,1.1077719,1.0424568,0.96409116,0.91385703,0.9255513,0.90369937,0.91716544,...,0.67574464,0.66714198,0.75205753,0.82026784,0.8957213,0.9829251,1.0491094,1.0931087,1.1025334,1.1120469
0,1,0.830904,0.815222,0.803198,0.742477,0.683805,0.672666,0.673233,0.679851,0.664352,...,0.545351,0.540754,0.603622,0.650685,0.708652,0.771576,0.791044,0.816584,0.819874,0.823568
1,1,0.816833,0.810941,0.772436,0.734036,0.695747,0.657560,0.617705,0.579676,0.541709,...,0.677791,0.709058,0.710436,0.729174,0.743558,0.785415,0.802844,0.805283,0.807960,0.810870
2,1,1.942775,1.928545,1.903324,1.879062,1.812798,1.746591,1.675465,1.611625,1.548000,...,1.660595,1.701224,1.748952,1.794030,1.845204,1.864267,1.860472,1.909628,1.958690,1.944351
3,1,1.873812,1.863549,1.814253,1.779266,1.766919,1.735888,1.675732,1.699785,1.720428,...,1.818965,1.854731,1.890964,1.918501,1.903307,1.910518,1.895921,1.881597,1.873381,1.898326
4,1,2.314335,2.302548,2.283370,2.246295,2.189444,2.150547,2.092030,2.038564,2.015111,...,1.970946,2.020198,2.072746,2.123032,2.172533,2.219684,2.267653,2.315809,2.317153,2.329100


In [8]:
#TODO: should I sort the column names?

**Conclusion: the features and the labels can be seperated by the column dtype**

In [9]:
# splitting the dataset to its features and labels
features = train_data.select_dtypes(include=['float64'])
labels = train_data.select_dtypes(include=['int64'])


In [10]:
# get the labels column name
print(labels.columns)

Index(['1'], dtype='object')


In [11]:
if PLOT:
    # get labels stats
    plt.figure(figsize=(15,3))
    ax = sns.countplot(y="1", data=labels)

    for p in ax.patches:
        ax.annotate(f'{p.get_width()}', (p.get_width()+1, p.get_y()+0.5))

**Conclusion: balanced dataset**

### Something with the column names doesn't make sense. maybe it should be sorted?

In [12]:
y = features.columns.astype("float")
x = np.arange(len(y))

In [13]:
if PLOT:
    fig, axes = plt.subplots(1,2, figsize=(10,5))
    axes[0].plot(x,y)
    axes[1].plot(x,sorted(y))

    axes[0].set_title("Original columns order")
    axes[1].set_title("Sorted columns order")

**Conclusion: Maybe it should be sorted... I should read the papers refferd to in the README.md file to better understand**

## Visualuziation

In [14]:
NUM_SAMPLES_PER_CLASS = 10

In [15]:
if PLOT:
    samples = visualize_dataset(train_data, NUM_SAMPLES_PER_CLASS)

**Conclusion: looks like a clustering \ knn algorithm will be usefull. I've noticed that some classes (e.g. 1) might have multiple sub-clusters**
**Conclusion: looks like a low pass filtering would be usefull**

In [16]:
if PLOT:
    rolling_dataset = pd.concat([labels, features.rolling(SMOOTHING_WINDOW, axis=1).mean()], axis=1)
    rolling_dataset = rolling_dataset.dropna(axis=1)
    rolling_samples = visualize_dataset(rolling_dataset, NUM_SAMPLES_PER_CLASS)

## Data Cleaning

In [17]:
# any dupplicates?
features.duplicated().any()

False

**Conclusion: No**

In [18]:
# any missing values?
features.isna().sum().sum()

0

**Conclusion: No**

In [19]:
# Low pass filtering
features = features.rolling(SMOOTHING_WINDOW, axis=1).mean()
features = features.dropna(axis=1)
    
features.shape

(499, 1024)

In [20]:
# downsampling
features = features[features.columns[::DOWNSAMPLE_WINDOW]]

features.shape

(499, 1024)

In [21]:
if PLOT:
    samples = visualize_dataset(pd.concat([labels, features], axis=1), NUM_SAMPLES_PER_CLASS)

## Remove outliers

# Data splitting

In [22]:
#TODO: do a kfold cross validation split


In [23]:
# splitting to train, val, test
X_train, X_test, y_train, y_test = train_test_split(features, labels, stratify=labels, test_size=0.1)#, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.22)#, random_state=1)


print("Train: ", np.unique(y_train, return_counts = True))
print("Val: ", np.unique(y_val, return_counts = True))
print("Test: ", np.unique(y_test, return_counts = True))

Train:  (array([1, 2, 3, 4, 5], dtype=int64), array([70, 70, 70, 70, 70], dtype=int64))
Val:  (array([1, 2, 3, 4, 5], dtype=int64), array([19, 20, 20, 20, 20], dtype=int64))
Test:  (array([1, 2, 3, 4, 5], dtype=int64), array([10, 10, 10, 10, 10], dtype=int64))


# Preprocessing

**Conclusion: all sequences have the same length, therefore there is no need for special care in this aspect**

In [24]:
#label encoding
# no need for a LabelEncoder(), just substruct 1 from the labels to have it 0-based.

# Feature extraction

# Modeling

## Experiment #1: KNN with DTW

* _

## Testing the DTW metric

In [25]:
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
# from fastdtw import fastdtw
# from scipy.spatial.distance import euclidean

In [40]:
class_names = y_train["1"].unique()

# dists = [ [ None ]*len(class_names) ]*len(class_names)

# for i in range(len(class_names)):

#     for j in range(i, len(class_names)):
#         cls_x = X_train.loc[y_train["1"]==class_names[i]]
#         cls_y = X_train.loc[y_train["1"]==class_names[j]]
#         dists[i][j] = [dtw.distance_fast(cls_x.iloc[x].values, cls_y.iloc[y].values, window=100) for x in range(5) for y in range(5)]



In [41]:
# fig, axes = plt.subplots(len(class_names), len(class_names), figsize=(15,10), sharey=True)

# for i in range(len(dists)):
#     for j in range(i, len(dists)):
#         sns.distplot(dists[j][i], ax=axes[j][i])
#         if i==0:
#             axes[i][j].set_title(f"Class: {class_names[j]}")
    

# plt.tight_layout()

In [43]:
# dists3 = np.zeros((len(cls1),len(cls1)))
# for i in tqdm(range(len(cls2))):
#     for j in range(len(cls2)):
#         s1 = cls2.iloc[i].values
#         s2 = cls2.iloc[j].values
#         dists3[i,j] = dtw.distance_fast(s1, s2)
# sns.distplot(dists3)
# sns.heatmap(dists3)

In [46]:
# nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric=dtw.distance_fast)
cls = KNeighborsClassifier(n_neighbors=1, metric=dtw.distance_fast, metric_params={"window": 100})
cls.fit(X_train, y_train["1"])
y_pred = cls.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

           1       1.00      0.79      0.88        19
           2       1.00      1.00      1.00        20
           3       0.95      0.90      0.92        20
           4       0.83      0.95      0.88        20
           5       0.91      1.00      0.95        20

    accuracy                           0.93        99
   macro avg       0.94      0.93      0.93        99
weighted avg       0.94      0.93      0.93        99

[[15  0  0  2  2]
 [ 0 20  0  0  0]
 [ 0  0 18  2  0]
 [ 0  0  1 19  0]
 [ 0  0  0  0 20]]


In [ ]:
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

In [51]:
cls = KNeighborsClassifier(n_neighbors=1, metric=dtw.distance_fast, metric_params={"window": 100})

In [52]:
cv_scores = cross_val_score(cls, 
                            features, 
                            labels["1"], 
                            cv=5, # integer, to specify the number of folds in a (Stratified)KFold,
                            scoring="f1_macro",
                            n_jobs=-1,
                            verbose=1,
                           )
cv_scores

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   35.8s remaining:   53.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   39.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   39.6s finished


array([0.88814779, 0.87939328, 0.85776446, 0.91676345, 0.89719776])

In [63]:
X = features
y = labels["1"]
k_scores = {}

skf = StratifiedKFold(n_splits=5)

for k, (train_index, val_index) in enumerate(skf.split(X, y)):
    k_scores[k] = {"score":0, "model":None}
    for n_neighbors in [1, 3]:
        for window in [100]:
            cls = KNeighborsClassifier(n_neighbors=n_neighbors, metric=dtw.distance_fast, metric_params={"window": window})
            print(cls)
            cls.fit(X.loc[train_index], y.loc[train_index])
            y_pred = cls.predict(X.loc[val_index])
            score = f1_score(y.loc[val_index], y_pred, average="macro")
            print(f"F1-score macro average: {score:.2f}")
            
            if score > k_scores[k]["score"]:
                k_scores[k]["score"]=score
                k_scores[k]["model"]=cls

KNeighborsClassifier(algorithm='auto', leaf_size=30,
                     metric=<function distance_fast at 0x0000022C03A91288>,
                     metric_params={'window': 100}, n_jobs=None, n_neighbors=1,
                     p=2, weights='uniform')
F1-score macro average: 0.89
KNeighborsClassifier(algorithm='auto', leaf_size=30,
                     metric=<function distance_fast at 0x0000022C03A91288>,
                     metric_params={'window': 100}, n_jobs=None, n_neighbors=3,
                     p=2, weights='uniform')
F1-score macro average: 0.91
KNeighborsClassifier(algorithm='auto', leaf_size=30,
                     metric=<function distance_fast at 0x0000022C03A91288>,
                     metric_params={'window': 100}, n_jobs=None, n_neighbors=1,
                     p=2, weights='uniform')
F1-score macro average: 0.88
KNeighborsClassifier(algorithm='auto', leaf_size=30,
                     metric=<function distance_fast at 0x0000022C03A91288>,
                     m

KeyboardInterrupt: 

In [65]:
k_scores

{0: {'score': 0.9083872737871603,
  'model': KNeighborsClassifier(algorithm='auto', leaf_size=30,
                       metric=<function distance_fast at 0x0000022C03A91288>,
                       metric_params={'window': 100}, n_jobs=None, n_neighbors=3,
                       p=2, weights='uniform')},
 1: {'score': 0.8793932768130459,
  'model': KNeighborsClassifier(algorithm='auto', leaf_size=30,
                       metric=<function distance_fast at 0x0000022C03A91288>,
                       metric_params={'window': 100}, n_jobs=None, n_neighbors=1,
                       p=2, weights='uniform')},
 2: {'score': 0.8577644577644576,
  'model': KNeighborsClassifier(algorithm='auto', leaf_size=30,
                       metric=<function distance_fast at 0x0000022C03A91288>,
                       metric_params={'window': 100}, n_jobs=None, n_neighbors=1,
                       p=2, weights='uniform')}}

In [960]:
grid_params = {
    "n_neighbors": [1,3,5,7,9],
    "weights": ["uniform", "distance"],
    "metric": [dtw.distance_fast, "minkowski"]
}

knn_gs = GridSearchCV(
    KNeighborsClassifier(),
    grid_params,
    verbose=1,
    cv=5,
    n_jobs=-1,
)

TypeError: distance_fast() missing 2 required positional arguments: 's1' and 's2'

In [ ]:
knn_gs_results = gs.fit(features, labels["1"])

In [ ]:
print("best_score: ", knn_gs_results.best_score_)
print("best_estimator_: ", knn_gs_results.best_estimator_)
print("best_params_: ", knn_gs_results.best_params_)

In [ ]:
scaler = preprocessing.MinMaxScaler()
features_scaled = scaler.fit_transform(features)


In [ ]:
knn_gs_results_scaled = knn_gs.fit(features, labels["1"])

## Experiment #2: Feature extraction with DWT + classic ML classifier

In [949]:
import pywt

In [ ]:
# CWT + CNN
# scales = np.arange(1,128)
# waveletname = 'mexh'
# sample = features.iloc[400]
# time = np.arange(len(sample.index))
# signal = sample.values
# plot_wavelet(time, signal, scales, waveletname=waveletname)

In [ ]:
DECONSTRUCT_LEVEL = 3

coeffs = pywt.wavedec(features.iloc[0], 'db2', level=DECONSTRUCT_LEVEL)
nth_level_approx_coeffs = coeffs[0]
nth_level_approx_coeffs.shape

In [ ]:
cwt_features = []
for i, row in features.iterrows():
    coeffs = pywt.wavedec(row, 'db2', level=DECONSTRUCT_LEVEL)
    nth_level_approx_coeffs = coeffs[0]
    cwt_features.append(nth_level_approx_coeffs)
    
cwt_features = pd.DataFrame(cwt_features, index=features.index)

In [ ]:
grid_params = {
    "max_depth": list(range(2,32,2)),
}

rf_gs = GridSearchCV(
    RandomForestClassifier(),
    grid_params,
    verbose=1,
    cv=5,
    n_jobs=-1,
)

In [ ]:
rf_gs_results = rf_gs.fit(cwt_features, labels["1"])

In [ ]:
print("best_score: ", rf_gs_results.best_score_)
print("best_estimator_: ", rf_gs_results.best_estimator_)
print("best_params_: ", rf_gs_results.best_params_)

In [ ]:
cwt_features_scaled = preprocessing.scale(cwt_features)
# cwt_features_scaled = cwt_features - cwt_features.mean()


In [ ]:
pca = PCA(n_components=2)
pca.fit(cwt_features_scaled)
X = pca.transform(cwt_features_scaled)

In [ ]:
cls1 = X[np.where(labels["1"]==1)[0]]
cls2 = X[np.where(labels["1"]==2)[0]]
cls3 = X[np.where(labels["1"]==3)[0]]
cls4 = X[np.where(labels["1"]==4)[0]]
cls5 = X[np.where(labels["1"]==5)[0]]

In [ ]:
plt.scatter(cls1[:, 0], cls1[:, 1], label=1)
plt.scatter(cls2[:, 0], cls2[:, 1], label=2)
plt.scatter(cls3[:, 0], cls3[:, 1], label=3)
plt.scatter(cls4[:, 0], cls4[:, 1], label=4)
plt.scatter(cls5[:, 0], cls5[:, 1], label=5)
plt.legend()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(cwt_features, labels_1["1"], stratify=labels, test_size=0.2)#, random_state=1)

In [ ]:
labels_1 = labels-1

xgb_model = xgb.XGBClassifier()

D_train = xgb.DMatrix(X_train, label=y_train)
D_val = xgb.DMatrix(X_val, label=y_val)

param = {
    'eta': 0.3, 
    'max_depth': 8,  
    'objective': 'multi:softprob',  
    'num_class': 5} 

# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

steps = 20  # The number of training iterations

model = xgb.train(param, D_train, steps)

y_pred = model.predict(D_val)
best_preds = np.asarray([np.argmax(line) for line in y_pred])

print(classification_report(y_val, best_preds))
print(confusion_matrix(y_val, best_preds))

## Experiment #3: Deep LSTM + FC classifier

In [167]:
from multiprocessing import cpu_count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from deep_ts import LSTMClassifier, Trainer

from utils import *

torch.manual_seed(1)

In [168]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using {device}")

Using cuda


In [169]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
y_enc = enc.fit_transform(labels["1"]-1)
(labels["1"].values-1).shape

(499,)

In [197]:
from sklearn.model_selection import train_test_split, StratifiedKFold


In [200]:
X_grouped = create_grouped_array(features)
y_enc = labels["1"].values-1

skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(X_grouped, y_enc)


2

In [218]:
print('Preparing datasets')
# trn_ds, val_ds, enc = create_datasets(features, labels["1"])
y_enc = labels["1"].values-1
# trn_ds, val_ds = create_datasets(features, y_enc)

trn_ds_list, val_ds_list = create_datasets_2(features, y_enc, cv=1)

batch_size = features.shape[0]
print(f'Creating data loaders with batch size: {bs}')
# trn_dl, val_dl = create_loaders(trn_ds, val_ds, bs) #, jobs=cpu_count())
data_loaders = [create_loaders(trn_ds, val_ds, batch_size) for trn_ds, val_ds in zip(trn_ds_list, val_ds_list)]

Preparing datasets
Creating data loaders with batch size: 499


In [244]:
k_scores = {}

for k, (trn_dl, val_dl) in enumerate(data_loaders):
    k_scores[k] = {"best_score":0, "model":None}
    for layer_dim in [1,2,3,4,5]:
        for hidden_dim in [16, 32, 64, 128, 256, 512]:
            input_dim = features.shape[1]
    #         hidden_dim = 128
    #         layer_dim = 1
            output_dim = 5

            n_epochs = 1000
            patience = 100
            lr = 0.0005
            iterations_per_epoch = len(trn_dl)

            model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
            model = model.to(device)
            criterion = nn.CrossEntropyLoss()
            opt = torch.optim.RMSprop(model.parameters(), lr=lr)
            sched = CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

            best_score = Trainer(model, trn_dl, val_dl, n_epochs, sched, opt, criterion, device, patience)
            
            if best_score > k_scores[k]["best_score"]:
                k_scores[k]["best_score"]=best_score
                k_scores[k]["model"]=model


Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=16, bias=True)
  (fc_out): Linear(in_features=16, out_features=5, bias=True)
)
Epoch 1 best model saved with f1_score: 11.88%
Epoch 2 best model saved with f1_score: 24.29%
Epoch 3 best model saved with f1_score: 35.47%
Epoch 4 best model saved with f1_score: 40.02%
Epoch 5 best model saved with f1_score: 42.31%
Epoch 6 best model saved with f1_score: 48.86%
Epoch 9 best model saved with f1_score: 50.10%
Epoch:  10. Loss: 1.4712. F1 score: 52.82%, Acc.: 55.00%
Epoch 10 best model saved with f1_score: 52.82%
Epoch 13 best model saved with f1_score: 55.94%
Epoch 14 best model saved with f1_score: 56.52%
Epoch 15 best model saved with f1_score: 60.96%
Epoch 16 best model saved with f1_score: 62.41%
Epoch 17 best model saved with f1_score: 64.95%
Epoch 18 best model saved with f1_score: 68.01%
Epoch 19 best model saved with f1_score: 68.90%
Epoch:  20. Loss: 1.3496. F1

Epoch:  80. Loss: 0.0352. F1 score: 89.84%, Acc.: 90.00%
Epoch 84 best model saved with f1_score: 91.82%
Epoch:  90. Loss: 0.0200. F1 score: 91.72%, Acc.: 92.00%
Epoch: 100. Loss: 0.0151. F1 score: 91.72%, Acc.: 92.00%
Epoch: 110. Loss: 0.0116. F1 score: 90.75%, Acc.: 91.00%
Epoch: 120. Loss: 0.0091. F1 score: 91.72%, Acc.: 92.00%
Epoch: 130. Loss: 0.0073. F1 score: 89.73%, Acc.: 90.00%
Epoch: 140. Loss: 0.0060. F1 score: 89.73%, Acc.: 90.00%
Epoch: 150. Loss: 0.0050. F1 score: 89.73%, Acc.: 90.00%
Epoch: 160. Loss: 0.0042. F1 score: 89.73%, Acc.: 90.00%
Epoch: 170. Loss: 0.0036. F1 score: 89.73%, Acc.: 90.00%
Epoch: 180. Loss: 0.0031. F1 score: 89.73%, Acc.: 90.00%
Early stopping on epoch 184
Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=256, bias=True)
  (fc_out): Linear(in_features=256, out_features=5, bias=True)
)
Epoch 1 best model saved with f1_score: 60.33%
Epoch 2 best model saved with f1_score: 66.3

Epoch 125 best model saved with f1_score: 87.54%
Epoch: 130. Loss: 0.3894. F1 score: 84.06%, Acc.: 85.00%
Epoch: 140. Loss: 0.3574. F1 score: 85.26%, Acc.: 86.00%
Epoch 145 best model saved with f1_score: 87.62%
Epoch: 150. Loss: 0.3228. F1 score: 87.54%, Acc.: 88.00%
Epoch: 160. Loss: 0.2889. F1 score: 86.29%, Acc.: 87.00%
Epoch 162 best model saved with f1_score: 88.54%
Epoch: 170. Loss: 0.2580. F1 score: 88.54%, Acc.: 89.00%
Epoch 176 best model saved with f1_score: 89.64%
Epoch 178 best model saved with f1_score: 90.73%
Epoch: 180. Loss: 0.2262. F1 score: 90.73%, Acc.: 91.00%
Epoch: 190. Loss: 0.1987. F1 score: 88.67%, Acc.: 89.00%
Epoch: 200. Loss: 0.1716. F1 score: 88.67%, Acc.: 89.00%
Epoch: 210. Loss: 0.1484. F1 score: 88.63%, Acc.: 89.00%
Epoch: 220. Loss: 0.1280. F1 score: 88.63%, Acc.: 89.00%
Epoch: 230. Loss: 0.1105. F1 score: 87.52%, Acc.: 88.00%
Epoch: 240. Loss: 0.0954. F1 score: 87.52%, Acc.: 88.00%
Epoch: 250. Loss: 0.0837. F1 score: 87.52%, Acc.: 88.00%
Epoch: 260. Lo

Epoch:  80. Loss: 0.0094. F1 score: 90.72%, Acc.: 91.00%
Epoch:  90. Loss: 0.0431. F1 score: 91.80%, Acc.: 92.00%
Epoch: 100. Loss: 0.0064. F1 score: 90.89%, Acc.: 91.00%
Epoch: 110. Loss: 0.0035. F1 score: 90.89%, Acc.: 91.00%
Epoch: 120. Loss: 0.0022. F1 score: 91.87%, Acc.: 92.00%
Epoch: 130. Loss: 0.0015. F1 score: 91.87%, Acc.: 92.00%
Epoch: 140. Loss: 0.0011. F1 score: 91.87%, Acc.: 92.00%
Epoch: 150. Loss: 0.0008. F1 score: 91.87%, Acc.: 92.00%
Epoch: 160. Loss: 0.0006. F1 score: 91.87%, Acc.: 92.00%
Early stopping on epoch 164
Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 16, num_layers=3, batch_first=True)
  (fc): Linear(in_features=16, out_features=16, bias=True)
  (fc_out): Linear(in_features=16, out_features=5, bias=True)
)
Epoch 1 best model saved with f1_score: 6.67%
Epoch:  10. Loss: 1.6122. F1 score: 6.67%, Acc.: 20.00%
Epoch:  20. Loss: 1.6036. F1 score: 6.67%, Acc.: 20.00%
Epoch:  30. Loss: 1.5913. F1 score: 16.25%, Acc.: 26.00%
Epoch 30 best model saved wi

Epoch 2 best model saved with f1_score: 47.30%
Epoch 3 best model saved with f1_score: 50.70%
Epoch:  10. Loss: 1.0478. F1 score: 21.97%, Acc.: 37.00%
Epoch 12 best model saved with f1_score: 52.13%
Epoch 14 best model saved with f1_score: 61.61%
Epoch 16 best model saved with f1_score: 65.61%
Epoch 18 best model saved with f1_score: 68.19%
Epoch:  20. Loss: 0.6545. F1 score: 70.46%, Acc.: 77.00%
Epoch 20 best model saved with f1_score: 70.46%
Epoch 22 best model saved with f1_score: 73.93%
Epoch:  30. Loss: 0.3957. F1 score: 69.61%, Acc.: 72.00%
Epoch 37 best model saved with f1_score: 74.69%
Epoch:  40. Loss: 0.3060. F1 score: 70.22%, Acc.: 77.00%
Epoch:  50. Loss: 0.2906. F1 score: 76.29%, Acc.: 79.00%
Epoch 50 best model saved with f1_score: 76.29%
Epoch 51 best model saved with f1_score: 79.29%
Epoch 59 best model saved with f1_score: 83.11%
Epoch:  60. Loss: 0.2658. F1 score: 81.87%, Acc.: 83.00%
Epoch 61 best model saved with f1_score: 84.31%
Epoch 65 best model saved with f1_sc

...

Epoch: 130. Loss: 0.0091. F1 score: 86.82%, Acc.: 86.87%
Epoch: 140. Loss: 0.0066. F1 score: 85.87%, Acc.: 85.86%
Epoch: 150. Loss: 0.0050. F1 score: 85.87%, Acc.: 85.86%
Epoch: 160. Loss: 0.0040. F1 score: 85.87%, Acc.: 85.86%
Epoch: 170. Loss: 0.0033. F1 score: 86.94%, Acc.: 86.87%
Epoch: 180. Loss: 0.0027. F1 score: 86.94%, Acc.: 86.87%
Epoch: 190. Loss: 0.0023. F1 score: 86.94%, Acc.: 86.87%
Epoch: 200. Loss: 0.0020. F1 score: 86.94%, Acc.: 86.87%
Early stopping on epoch 208
Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 256, num_layers=3, batch_first=True)
  (fc): Linear(in_features=256, out_features=256, bias=True)
  (fc_out): Linear(in_features=256, out_features=5, bias=True)
)
Epoch 1 best model saved with f1_score: 23.66%
Epoch 2 best model saved with f1_score: 56.27%
Epoch 3 best model saved with f1_score: 63.73%
Epoch 4 best model saved with f1_score: 64.94%
Epoch 6 best model saved with f1_score: 65.54%
Epoch 8 best model saved with f1_score: 70.19%
Epoch:  10. Lo

Epoch: 410. Loss: 0.7895. F1 score: 50.29%, Acc.: 59.60%
Epoch: 420. Loss: 0.7707. F1 score: 49.67%, Acc.: 59.60%
Epoch: 430. Loss: 0.7535. F1 score: 49.68%, Acc.: 59.60%
Epoch: 440. Loss: 0.7385. F1 score: 51.22%, Acc.: 60.61%
Epoch 441 best model saved with f1_score: 54.43%
Epoch 443 best model saved with f1_score: 55.37%
Epoch 445 best model saved with f1_score: 58.78%
Epoch: 450. Loss: 0.7217. F1 score: 56.05%, Acc.: 62.63%
Epoch 454 best model saved with f1_score: 60.68%
Epoch 456 best model saved with f1_score: 60.78%
Epoch 458 best model saved with f1_score: 63.37%
Epoch: 460. Loss: 0.7038. F1 score: 62.48%, Acc.: 66.67%
Epoch: 470. Loss: 0.6890. F1 score: 62.06%, Acc.: 65.66%
Epoch 473 best model saved with f1_score: 64.27%
Epoch 476 best model saved with f1_score: 64.95%
Epoch 477 best model saved with f1_score: 65.74%
Epoch: 480. Loss: 0.6774. F1 score: 64.95%, Acc.: 68.69%
Epoch 483 best model saved with f1_score: 67.50%
Epoch 485 best model saved with f1_score: 68.99%
Epoch

Epoch: 120. Loss: 0.0500. F1 score: 88.38%, Acc.: 88.89%
Epoch: 130. Loss: 0.0647. F1 score: 85.38%, Acc.: 85.86%
Epoch: 140. Loss: 0.0481. F1 score: 88.28%, Acc.: 88.89%
Epoch: 150. Loss: 0.0436. F1 score: 88.31%, Acc.: 88.89%
Epoch: 160. Loss: 0.0410. F1 score: 88.31%, Acc.: 88.89%
Epoch: 170. Loss: 0.0390. F1 score: 87.35%, Acc.: 87.88%
Epoch: 180. Loss: 0.0373. F1 score: 87.35%, Acc.: 87.88%
Epoch: 190. Loss: 0.0358. F1 score: 87.42%, Acc.: 87.88%
Early stopping on epoch 193
Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 256, num_layers=4, batch_first=True)
  (fc): Linear(in_features=256, out_features=256, bias=True)
  (fc_out): Linear(in_features=256, out_features=5, bias=True)
)
Epoch 1 best model saved with f1_score: 6.72%
Epoch 2 best model saved with f1_score: 24.07%
Epoch 3 best model saved with f1_score: 27.20%
Epoch 4 best model saved with f1_score: 32.43%
Epoch:  10. Loss: 1.0603. F1 score: 46.38%, Acc.: 51.52%
Epoch 10 best model saved with f1_score: 46.38%
Epoc

Epoch:  90. Loss: 0.3022. F1 score: 65.87%, Acc.: 71.72%
Epoch: 100. Loss: 0.2986. F1 score: 65.87%, Acc.: 71.72%
Epoch: 110. Loss: 0.2880. F1 score: 66.25%, Acc.: 71.72%
Epoch: 120. Loss: 0.2858. F1 score: 65.10%, Acc.: 70.71%
Epoch: 130. Loss: 0.2833. F1 score: 65.10%, Acc.: 70.71%
Epoch 138 best model saved with f1_score: 76.79%
Epoch: 140. Loss: 0.2799. F1 score: 73.83%, Acc.: 76.77%
Epoch 144 best model saved with f1_score: 78.16%
Epoch: 150. Loss: 0.5601. F1 score: 78.72%, Acc.: 79.80%
Epoch 150 best model saved with f1_score: 78.72%
Epoch 159 best model saved with f1_score: 79.72%
Epoch: 160. Loss: 0.2767. F1 score: 78.45%, Acc.: 79.80%
Epoch 161 best model saved with f1_score: 80.93%
Epoch 163 best model saved with f1_score: 81.08%
Epoch 167 best model saved with f1_score: 81.81%
Epoch 168 best model saved with f1_score: 82.38%
Epoch: 170. Loss: 0.2469. F1 score: 75.83%, Acc.: 77.78%
Epoch 172 best model saved with f1_score: 82.54%
Epoch 175 best model saved with f1_score: 82.5

Epoch: 310. Loss: 0.0012. F1 score: 89.88%, Acc.: 89.90%
Epoch: 320. Loss: 0.0008. F1 score: 89.88%, Acc.: 89.90%
Early stopping on epoch 327
{0: {'best_score': 0.9491912708600772, 'model': 0.9491912708600772}, 1: {'best_score': 0.9296289504248426, 'model': 0.9296289504248426}, 2: {'best_score': 0.8893941702285734, 'model': 0.8893941702285734}, 3: {'best_score': 0.9397109051107918, 'model': 0.9397109051107918}, 4: {'best_score': 0.9381727574750831, 'model': 0.9381727574750831}}
Mean F1 Score:  0.9292196108198736


In [246]:
display(k_scores)
print("Mean F1 Score: ", np.mean([item["best_score"] for k, item in k_scores.items()]))

{0: {'best_score': 0.9491912708600772, 'model': 0.9491912708600772},
 1: {'best_score': 0.9296289504248426, 'model': 0.9296289504248426},
 2: {'best_score': 0.8893941702285734, 'model': 0.8893941702285734},
 3: {'best_score': 0.9397109051107918, 'model': 0.9397109051107918},
 4: {'best_score': 0.9381727574750831, 'model': 0.9381727574750831}}

Mean F1 Score:  0.9292196108198736


In [ ]:
print('The training is finished! Restoring the best model weights')

model.load_state_dict(torch.load('best.pth'))

In [ ]:
model.eval()

In [ ]:
a = preds
a = [a for i in range(10)]

In [ ]:
y_val